In [ ]:
from PyKomoran import *

In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
dic = pd.read_csv('result_file.csv',encoding = 'UTF-8-sig')
comment_data = dic[['comment','label']]

In [ ]:
#한글과 공백을 제외하고 모두 제거
normalized_text = []
for string in X_data.tolist():
    try:
        tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", string.lower())
    except Exception as e:
        print(string)
        break
    normalized_text.append(tokens)

In [ ]:
comment_data.comment = normalized_text

In [ ]:
#불용어 제거
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
kmr = Komoran("EXP")

In [ ]:
X_token=[]
for sentence in comment_data['comment']:
    temp_X = []
    temp_X = kmr.morphs(sentence) # 토큰화
    #temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_token.append(temp_X)

# 정수 인코딩 수행

In [ ]:
max_words = 35000
tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
tokenizer.fit_on_texts(X_token) 
X_token = tokenizer.texts_to_sequences(X_token)

In [ ]:
word_to_index = tokenizer.word_index

In [ ]:
vocab_size = len(word_to_index)+1

# 전체 데이터 길이 분포 확인

In [ ]:
print('train 리뷰의 최대 길이 :',max(len(l) for l in X_token))
print('리뷰의 평균 길이 :',sum(map(len, X_token))/len(X_token))
plt.hist([len(s) for s in X_token], bins=100)
plt.xlabel('length of Data')
plt.ylabel('number of Data')
plt.show()

In [ ]:
max_len = 124
X_data = pad_sequences(X_token, maxlen=max_len)
print("data shape: ", X_data.shape)

In [ ]:
y_data = np.array(y_data).reshape(-1, 1)

In [ ]:
print(type(X_data))
print(type(y_data))

In [ ]:
print(X_data[:3])
print(y_data[:3])

# 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size= 0.3, random_state=1234)

In [ ]:
print(X_train[:3])
print(y_train[:3]) 

# 모델 만들기

In [ ]:
import gensim
model = gensim.models.Word2Vec.load('D:\\Users\\Desktop\\GRAD\\grad_project\\preprocessing\\ko\\ko.bin')

In [ ]:
result = model.wv.most_similar("강아지")
print(result)

In [ ]:
embedding_matrix = np.zeros((vocab_size, 200))# 단어 집합 크기의 행과 200개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

In [ ]:
def get_vector(word):
    if word in model:
        return model[word]
    else:
        return None

In [ ]:
for word, i in tokenizer.word_index.items(): # 훈련 데이터의 단어 집합에서 단어와 정수 인덱스를 1개씩 꺼내온다.
    temp = get_vector(word) # 단어(key) 해당되는 임베딩 벡터의 300개의 값(value)를 임시 변수에 저장
    if temp is not None: # 만약 None이 아니라면 임베딩 벡터의 값을 리턴받은 것이므로
        embedding_matrix[i] = temp # 해당 단어 위치의 행에 벡터의 값을 저장한다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 200, weights = [embedding_matrix], input_length = max_len, trainable = False)

In [ ]:
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=5, verbose=1, validation_split = 0.2)

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))